In [50]:
import numpy as np
import cv2 
import mediapipe as mp
mp_draw=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

In [51]:
def detection(img,model):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img.flags.writeable=False
    results=model.process(img)
    img.flags.writeable=True
    img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    return img,results


In [84]:
def draw_edit(landmarks,frame):
    h,w,_=frame.shape
    nose=np.array([[landmarks[mp_pose.PoseLandmark.NOSE].x*w,landmarks[mp_pose.PoseLandmark.NOSE].y*h]])
    lft_shoulder=np.array([[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x*w,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y*h]])
    rght_shoulder=np.array([[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x*w,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y*h]])
    neck=((lft_shoulder+rght_shoulder)/2+nose)/2
    chest_mid=(lft_shoulder+rght_shoulder)/2
    x1,y1=np.squeeze(neck)
    x2,y2=np.squeeze(chest_mid)
    cv2.circle(frame, (int(x1), int(y1)), 6, (0,255,0), -1)
    cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255,255,255), 4)
    

In [85]:
cap=cv2.VideoCapture('VIDEO_TEST/2.mp4')#path chon video capture
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5,model_complexity=1) as pose:
    while cap.isOpened():
        ret,frame=cap.read()
        img,results=detection(frame,pose)
        mp_draw.draw_landmarks(img,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)
        try:
            landmark=results.pose_landmarks.landmark
            draw_edit(landmark,img)
        except:
            pass
        # mp_draw.draw_landmarks(img,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)
        if cv2.waitKey(20)& 0xFF==ord('q'):
            break
        cv2.imshow('Pose',img)
cap.release()
cv2.destroyAllWindows()

528.768185377121